## Import Modules

In [1]:
import dataset as ds
import config as cfg
import tools

import torch
from ultralytics import settings
from ultralytics import YOLO

import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

from clearml import Task
import clearml
clearml.browser_login()
import gc


# ClearML
https://app.clear.ml/dashboard

# Yolo Config

In [2]:
# Ultralytics settings
settings.reset()
settings.update({'datasets_dir': cfg.YOLO_DATASET_DIR.replace('/','\\')})
print(settings)

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")
print(torch.backends.cudnn.version())

#Disable Data Augmentation (potentially)
#hsv_h: 0.0  # hue
#hsv_s: 0.0  # saturation
#hsv_v: 0.0  # value
#degrees: 0.0  # rotation
#translate: 0.0  # translate
#scale: 0.0  # scale
#shear: 0.0  # shear
#perspective: 0.0  # perspective
#flipud: 0.0  # flip up-down
#fliplr: 0.0  # flip left-right
#mosaic: 0.0  # mosaic
#mixup: 0.0  # mixup


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\YOLOv8', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
False
None


## Load Dataset

In [3]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

#train_df.info()

In [4]:
# augmentation metadata

#augmentation_metadata = {
#    'methods': {
#        'hsv': {
#            'parameters': {
#                'type': 'h',  # Probability of applying the augmentation
#                'shift': 50,
#                'p': 1.0
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'rotate': {
#            'parameters': {
#                'angle': 90,  # Rotation angle
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 30% of the training images
#        },
#        'bnc': {
#            'parameters': {
#                'alpha': 1.5,  # contrast control
#                'beta': 50, # brightness control
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'histEq': {
#            'parameters': {
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'whiteBal': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'sharpen': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'gaussian': {
#            'parameters': {
#                'var_limit': (10.0, 50.0) # range of noise varaince
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'zoom': {
#            'parameters': {
#                'zoom_factor': 3  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        }       
#    }
#}


# Test Parameters

In [5]:
# name dataset to be create as well as task name
dataset_name = 'test500'

# project and task constant parameters
project_name= cfg.YOLO_PROJECT_NAME
dataset_dir = f'{cfg.YOLO_DATASET_DIR}/{dataset_name}'
project_dir = f'{cfg.YOLO_PROJECT_DIR}/{dataset_name}/'
class_names = cfg.YOLO_CLASS_NAMES

# task specifc training parameters
epochs = 20
model_variant = "yolov8n"

"""
Methods: 
 - flip
 - rotate
 - bnc
 - gaussian
 - histEq
 - whiteBal
 - sharpen
 - zoom
 - hsv
"""

augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

# Create Datasets

In [6]:
from sklearn.model_selection import train_test_split

_, test_train_df = train_test_split(
    train_df,
    test_size=500,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=100,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

# create sub dataset
ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)

# augment dataset
ds.augment_dataset(dataset_dir, augmentation_metadata)

Removing dataset if pre-existing


KeyboardInterrupt: 

# Training

In [ ]:
# train on pure dataset

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}.yaml'
project =  project_dir + 'pure' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-pure"
)
task.set_parameter("model_variant", model_variant)

# Define Yolo model
#model = YOLO(f'{model_variant}.yaml') # train on model which is not pre-trained
model = YOLO(f'{model_variant}.pt')

#train args
args = dict(data=dataset_path, 
            epochs=epochs, 
            device=0, 
            #close_mosaic=epochs, # disable mosaic augmentation
            seed=42
            )
task.connect(args)

# train model
results=model.train(**args, project=project)

# validate model
metrics = model.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category
metrics.box.mp    # P
metrics.box.mr    # R

# close task for next run
task.close()

#clear variables from memory
task = None
model = None
args = None
results = None


ClearML Task: created new task id=3da4a393ebf94ecfb10bb41484c2eebf
2024-05-01 23:25:23,844 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/0e6b6f2d2ca946d584a5db9d951794d1/experiments/3da4a393ebf94ecfb10bb41484c2eebf/output/log
2024-05-01 23:25:29,293 - clearml.model - INFO - Selected model id: 6932d54ba60d49f7bd3432493468f8c1
New https://pypi.org/project/ultralytics/8.2.6 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test500\test500.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test500/pure, name=train19, exist_ok=False, pr

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1650.57it/s]

train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 403.91it/s]

val: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid.cache


Plotting labels to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train19\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train19
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      2.14G       2.94      5.819      2.012         12        640: 100%|██████████| 32/32 [00:13<00:00,  2.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 1/4 [00:00<00:02,  1.07it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
ClearML Monitor: Reporting detected, reverting back to iteration based reporting


In [ ]:
print(metrics.box.map  )  # map50-95
print(metrics.box.map50)  # map50
print(metrics.box.map75)  # map75
print(metrics.box.maps )  # a list contains map50-95 of each category
print(metrics.box.mp   ) # P
print(metrics.box.mr   ) # R

In [ ]:
# train on augmented dataset

# clear variables
torch.cuda.empty_cache()
gc.collect()

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}-aug.yaml'
project =  project_dir + 'augmented' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-augmented"
)
task.set_parameter("model_variant", model_variant) 

# Define Yolo model
#model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

# train model
results = model.train(data=dataset_path, 
                      epochs=epochs, 
                      device=0, 
                      project=project,
                      #close_mosaic=epochs, # disable mosaic augmentation
                      seed=42)

# validate model
metrics = model.val()
metrics = model.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category
metrics.box.mp    # P
metrics.box.mr    # R

task.upload_artifact('AUGMENTATION_METADATA', tools.pretty_print_dict(augmentation_metadata))


# close task for next/last run
task.close()



In [ ]:
print(metrics.box.map  )  # map50-95
print(metrics.box.map50)  # map50
print(metrics.box.map75)  # map75
print(metrics.box.maps )  # a list contains map50-95 of each category
print(metrics.box.mp   ) # P
print(metrics.box.mr   ) # R